### Convertir un notebook en script

Si vous avez utilisé des notebooks pour l’expérimentation et le développement, vous devez d’abord convertir un notebook en script. Vous pouvez également choisir de ne pas utiliser de notebooks et de travailler uniquement avec des scripts. Dans les deux cas, pour préparer votre code pour la production, vous devez appliquer certaines recommandations lors de la création de scripts.

Les scripts conviennent parfaitement aux tests et à l’automatisation dans votre environnement de production. Pour créer un script prêt pour la production, vous devez :

### Supprimer le code non essentiel ;
- refactoriser votre code en fonctions ;
- tester votre script dans le terminal.
- Supprimer tout le code non essentiel
Le principal avantage de l’utilisation de notebooks est de pouvoir explorer rapidement des données. Par exemple, vous pouvez utiliser les instructions print() et df.describe() pour explorer vos données et vos variables. Quand vous créez un script qui sera utilisé pour l’automatisation, évitez d’inclure du code écrit à des fins exploratoires.

La première chose que vous devez donc faire pour convertir votre code en code de production est de supprimer le code non essentiel. En particulier, quand vous exécutez le code régulièrement, omettez tout ce qui n’est pas essentiel pour réduire les coûts et le temps de calcul.

### Refactoriser le code en fonctions
Quand vous utilisez du code dans des processus métier, le code doit être facile à lire pour que tout le monde puisse le gérer. Une approche courante pour rendre le code plus facile à lire et à tester consiste à utiliser des fonctions.

Par exemple, vous avez peut-être utilisé l’exemple de code suivant dans un notebook pour lire et fractionner les données

In [ ]:
# read and visualize the data
print("Reading data...")
df = pd.read_csv('diabetes.csv')
df.head()

# split data
print("Splitting data...")
X, y = df[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, df['Diabetic'].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

Dans la mesure où les fonctions permettent également de tester des parties de votre code, vous pouvez créer plusieurs fonctions plus petites plutôt qu’une grande fonction. Si vous souhaitez tester une partie de votre code, vous pouvez choisir de tester uniquement une petite partie en veillant à ne pas exécuter plus de code que nécessaire.

Vous pouvez refactoriser le code présenté dans l’exemple en deux fonctions :

Lire les données
Fractionner les données
Voici un exemple de code refactorisé :

In [ ]:
def main(csv_file):
    # read data
    df = get_data(csv_file)

    # split data
    X_train, X_test, y_train, y_test = split_data(df)

# function that reads the data
def get_data(path):
    df = pd.read_csv(path)
    
    return df

# function that splits the data
def split_data(df):
    X, y = df[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness',
    'SerumInsulin','BMI','DiabetesPedigree','Age']].values, df['Diabetic'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

    return X_train, X_test, y_train, y_test

## Exécuter un script en tant que travail de commande

Quand vous avez un script qui entraîne un modèle Machine Learning, vous pouvez l’exécuter en tant que travail de commande dans Azure Machine Learning.

### Configurer et soumettre un travail de commande
Pour exécuter un script en tant que travail de commande, vous devez configurer et soumettre le travail.

Pour configurer un travail de commande avec le SDK Python (v2), vous utiliserez la fonction command. Pour exécuter un script, vous devez spécifier des valeurs pour les paramètres suivants :

- `code`: dossier contenant le script à exécuter.
- `command`: spécifie le fichier à exécuter.
- `environment`: packages nécessaires à installer sur le calcul avant d’exécuter la commande.
- `compute`: calcul à utiliser pour exécuter la commande.
- `display_name`: nom du travail individuel.
- `experiment_name`: nom de l’expérience à laquelle le travail appartient.

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python train.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="train-model",
    experiment_name="train-classification-model"
    )

Une fois votre travail configuré, vous pouvez le soumettre, ce qui a pour effets de le lancer et d’exécuter le script :
```python
# submit job
returned_job = ml_client.create_or_update(job)
```


## Utiliser des paramètres dans un travail de commande

Vous pouvez augmenter la flexibilité de vos scripts en utilisant des paramètres. Par exemple, vous avez peut-être créé un script qui entraîne un modèle Machine Learning. Vous pouvez utiliser le même script pour entraîner un modèle sur différents jeux de données ou à l’aide de différentes valeurs d’hyperparamètres.

### Utilisation d’arguments de script
Pour utiliser des paramètres dans un script, vous devez utiliser une bibliothèque telle que argparse pour lire les arguments passés au script et les affecter à des variables.

Par exemple, le script suivant lit un argument nommé training_data qui spécifie le chemin aux données d’entraînement.

In [ ]:
# import libraries
import argparse
import pandas as pd
from sklearn.linear_model import LogisticRegression

def main(args):
    # read data
    df = get_data(args.training_data)

# function that reads the data
def get_data(path):
    df = pd.read_csv(path)
    
    return df

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", dest='training_data',
                        type=str)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":

    # parse args
    args = parse_args()

    # run main function
    main(args)

### Passage d’arguments à un script
Pour passer des valeurs de paramètre à un script, vous devez fournir la valeur de l’argument dans la commande.

Par exemple, si vous passez une valeur de paramètre lors de l’exécution d’un script dans un terminal, utilisez la commande suivante :

```python
python train.py --training_data diabetes.csv
```

Dans l’exemple, `diabetes.csv` est un fichier local. Vous pouvez également spécifier le chemin à une ressource de données créée dans l’espace de travail Azure Machine Learning.

De même, pour passer une valeur de paramètre à un script que vous souhaitez exécuter en tant que travail de commande, vous devez spécifier les valeurs dans la commande :

```python
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python train.py --training_data diabetes.csv",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="train-model",
    experiment_name="train-classification-model"
    )
```